#  会話型インターフェイス - Claude LLM 搭載チャットボット

> *このノートブックは SageMaker Studio の **`Data Science 3.0`** カーネルをご利用ください*

このノートブックでは、Amazon Bedrock の基本モデル (FM) を使用してチャットボットを構築します。このユースケースでは、チャットボットを構築するための FM として Claude を使用しています。

## 概要

チャットボットやバーチャルアシスタントなどの会話型インターフェースを使用して、顧客のユーザーエクスペリエンスを向上させることができます。チャットボットは、自然言語処理 (NLP) と機械学習アルゴリズムを使用してユーザーのクエリを理解し、それに応答します。チャットボットは、顧客サービス、営業、電子商取引などのさまざまなアプリケーションで使用可能で、ユーザーに迅速かつ効率的に対応することができます。ウェブサイト、ソーシャルメディアプラットフォーム、メッセージングアプリなど、さまざまなチャネルからアクセスできます。

## Amazon Bedrock を利用したチャットボット

![Amazon Bedrock - Conversational Interface](./images/chatbot_bedrock.png)


## ユースケース
1. **チャットボット(基本)** -  FM モデルの Zero-Shot チャットボット
2. **プロンプトテンプレートを使用したチャットボット** - プロンプトテンプレートにコンテキストが提供されているチャットボット
3. **ペルソナチャットボット** - 役割が定義されたチャットボット。例：キャリアコーチとヒューマンインタラクション
4. **コンテキストを意識したチャットボット** - 埋め込み生成による外部ファイルを利用したコンテキストを意識したチャットボット

## Amazon Bedrock でチャットボットを構築するための Langchain フレームワーク
チャットボットのような会話型インターフェースでは、短期的にも長期的にも、以前のやりとりを覚えておくことが非常に重要です。

LangChain は2つの形式のメモリコンポーネントを提供します。まず、LangChain は以前のチャットメッセージを管理および操作するためのヘルパーユーティリティを提供します。これらはモジュラーで、使用方法に関係なく役立つように設計されています。次に、LangChain はこれらのユーティリティをチェーンに組み込む簡単な方法を提供します。
これにより、さまざまなタイプの抽象化を簡単に定義して操作できるようになるため、強力なチャットボットの構築が容易になります。

## コンテキストを利用したチャットボットの構築-主な要素
コンテキストを意識したチャットボットを構築する最初のプロセスは、コンテキストの**埋め込みを生成**することです。通常、埋め込み (Embedding) モデルを実行して埋め込みを生成し、それをある種のベクトルストアに保存する取り込みプロセスがあります。この例では、Titan Embeddings モデルを使用しています。

![Embeddings](./images/embeddings_lang.png)

2 番目のプロセスは、ユーザーリクエストのオーケストレーション、インタラクション、結果の呼び出しと出力です。

![Chatbot](./images/chatbot_lang.png)

## アーキテクチャ [コンテクストを意識したチャットボット]
![4](./images/context-aware-chatbot.png)


## セットアップ

このノートブックの残りの部分を実行する前に、以下のセルを実行して (必要なライブラリがインストールされていることを確認し) Bedrock に接続する必要があります。

セットアップの仕組みと ⚠️ **whether you might need to make any changes** についての詳細は、[Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ja.ipynb) を参照してください。

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

この Notebook では、追加の依存関係が必要になります：

- [FAISS](https://github.com/facebookresearch/faiss)...　埋め込みベクトルを保存するため
- [IPyWidgets](https://ipywidgets.readthedocs.io/en/stable/)... Notebook のインタラクティブな UI ウィジェットのため
- [PyPDF](https://pypi.org/project/pypdf/)... PDF ファイルを操作するため

In [ ]:
%pip install --quiet "faiss-cpu>=1.7,<2" "ipywidgets>=7,<8" langchain==0.0.309 "pypdf>=3.8,<4"

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ 必要に応じて AWS 設定に関する以下のコードのコメントを解除、編集してください ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

### チャットボット (基本 - コンテキストなし)

私たちは [CoverSationChain](https://python.langchain.com/en/latest/modules/models/llms/integrations/bedrock.html?highlight=ConversationChain#using-in-a-conversation-chain)を使用して、 LangChain から会話を始めます。また、メッセージの保存には [ConversationBufferMemory](https://python.langchain.com/en/latest/modules/memory/types/buffer.html) を使用します。履歴をメッセージのリストとして取得することもできます（これはチャットモデルでは非常に便利です）。

チャットボットは以前のやりとりを覚えておく必要があります。会話型メモリはそれを可能にします。会話型メモリを実装する方法はいくつかあります。LangChain のコンテキストでは、これらはすべて ConversationChain の上に構築されています。

**注意:** モデルの出力は非決定的です

In [ ]:
from langchain.chains import ConversationChain
from langchain.llms.bedrock import Bedrock
from langchain.memory import ConversationBufferMemory
modelId = "anthropic.claude-v2"
cl_llm = Bedrock(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs={"max_tokens_to_sample": 1000},
)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=cl_llm, verbose=True, memory=memory
)

try:
    
    print_ww(conversation.predict(input="Hi there!"))

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

ここで何が起こっているのでしょうか？我々は「Hi there！」と言い、モデルは何度か会話を交わしました。これは、Langchain の ConversationChain が使用するデフォルトのプロンプトが、Claude 向けにうまく設計されていないためです。[効果的な Claude のプロンプト](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design) は先頭に `\n\nHuman` を含み、後に`\n\nAassistant:` を含むべきだとされています。(オプションとして後続にテキストも含むことができるようです。[詳しくはこちらをご参照ください](https://docs.anthropic.com/claude/docs/human-and-assistant-formatting#use-human-and-assistant-to-put-words-in-claudes-mouth)) では修正しましょう。

Claude のプロンプトの書き方について詳しくは、[Anthropic documentation](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design) をご覧ください。

## プロンプトテンプレートを使用したチャットボット (Langchain)

LangChain には、プロンプトの作成と操作を簡単にするためのクラスと関数がいくつか用意されています。[PromptTemplate](https://python.langchain.com/en/latest/modules/prompts/getting_started.html) クラスを使用して f-string テンプレートからプロンプトを作成します。 

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain import PromptTemplate

# turn verbose to true to see the full logs and documents
conversation= ConversationChain(
    llm=cl_llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

# langchain prompts do not always work with all the models. This prompt is tuned for Claude
claude_prompt = PromptTemplate.from_template("""

Human: The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context. If the AI does not know
the answer to a question, it truthfully says it does not know.

Current conversation:
<conversation_history>
{history}
</conversation_history>

Here is the human's next reply:
<human_reply>
{input}
</human_reply>

Assistant:
""")

conversation.prompt = claude_prompt

print_ww(conversation.predict(input="Hi there!"))

#### 新しい質問

モデルは最初のメッセージに応答しました。次の質問をしてみましょう。

In [ ]:
print_ww(conversation.predict(input="Give me a few tips on how to start a new garden."))

#### 質問を構築する

garden という言葉は用いずに質問をして、モデルが以前の会話を理解できるか確認してみましょう

In [ ]:
print_ww(conversation.predict(input="Cool. Will that work with tomatoes?"))

#### 会話を終了する

In [ ]:
print_ww(conversation.predict(input="That's all, thank you!"))

Claude は本当におしゃべりです。プロンプトを変更して、短い回答をさせてみてください。

### ipywidget を用いたインタラクティブセッション

次のユーティリティクラスを使用すると、より自然に Claude と対話できます。入力ボックスに質問を書き込んで、Claude に答えてもらいます。その後は会話を続けることができます

In [ ]:
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            print("Thank you , that was a nice chat!!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        result = self.qa.run({'question': prompt })
                    else:
                        result = self.qa.run({'input': prompt }) #, 'history':chat_history})
                except:
                    result = "No answer"
                thinking.value=""
                print_ww(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

チャットを始めましょう。次の質問を試すこともできます。

1. tell me a joke
2. tell me another joke
3. what was the first joke about
4. can you make another joke on the same topic of the first joke

In [ ]:
chat = ChatUX(conversation)
chat.start_chat()

## ペルソナチャットボット

AI アシスタントはキャリアコーチの役割を果たします。ロールプレイダイアログでは、チャットを開始する前にユーザーメッセージを設定する必要があります。会話バッファメモリはダイアログの事前入力に使用されます。

In [ ]:
# ConversationalBufferMemory を使用して以前のやり取りを保存し、チャットにカスタムプロンプトを追加します。
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("You will be acting as a career coach. Your goal is to give career advice to users")
memory.chat_memory.add_ai_message("I am a career coach and give career advice")
cl_llm = Bedrock(model_id="anthropic.claude-v2",client=boto3_bedrock)
conversation = ConversationChain(
     llm=cl_llm, verbose=True, memory=memory
)

conversation.prompt = claude_prompt

print_ww(conversation.predict(input="What are the career options in AI?"))

In [ ]:
print_ww(conversation.predict(input="What these people really do? Is it fun?"))

##### このペルソナの専門ではない質問をしてみましょう。モデルはその質問に答えたりその理由を述べたりするべきではありません。

In [ ]:
conversation.verbose = False
print_ww(conversation.predict(input="How to fix my car?"))

## コンテキストを意識したチャットボット
このユースケースでは、チャットボットがこれまで見たことがないと思われる外部コーパスからの質問に回答するようにします。そのために、RAG (拡張検索生成) と呼ばれるパターンを適用します。コーパスをチャンク単位でインデックス付けし、チャンクと質問の間の意味的な類似性を利用して、コーパスのどのセクションに関連性があるかを調べて答えを出すという手法です。最後には、最も関連性の高いチャンクが集約され、コンテキストとして ConversationChain に履歴の提供と同様に渡されます。

csvファイルを取得して、**Titan Embeddings モデル** を使用して csv の各行のベクトルを作成します。次に、このベクトルは、インメモリベクトルデータストアを提供するオープンソースライブラリである FAISS に保存されます。チャットボットに質問が行われると、その質問で FAISS にクエリを行い、意味的に最も近いテキストを取得します。これが回答になります。

#### Titan Embeddings モデル


埋め込みは、単語、フレーズ、またはその他の個別の項目を連続ベクトル空間のベクトルとして表現する方法です。これにより、機械学習モデルはこれらの表現に対して数学的な操作を実行し、それらの間の意味的な関係を捉えることができます。


こちらは RAG に使用されます。 [document search capability](https://labelbox.com/blog/how-vector-similarity-search-works/) 

In [ ]:
from langchain.embeddings import BedrockEmbeddings
modelId = "amazon.titan-embed-g1-text-02"
#modelId = "amazon.titan-embed-text-v1"
br_embeddings = BedrockEmbeddings(model_id=modelId, client=boto3_bedrock)

#### ベクトルストアインデクサ

これが埋め込みデータを保存し、マッチさせるものです。このノートブックには Chroma と FAISS が搭載されており、一時的にメモリに保存れます。ベクトルストア API は [こちら](https://python.langchain.com/en/harrison-docs-refactor-3-24/reference/modules/vectorstore.html) で入手できます。

SageMaker Embeddings の独自のカスタム実装を使用します。この実装では、埋め込みを行うモデルを呼び出すために SageMaker エンドポイントへの参照が必要です。これは FAISS や Chroma がメモリに保存するのに使用され、ユーザーがクエリを実行するたびに使われます。

#### ベクトルストアとしての FAISS

検索に埋め込みを利用できるようにするには、ベクトル検索を効率的に実行できるストアが必要です。このノートブックでは、インメモリストアである FAISS を使用しています。ベクトルを恒久的に保存するには、pgVector、Pinecone、また Chroma を使用できます。

Langchain ベクトルストア API は[こちら](https://python.langchain.com/en/harrison-docs-refactor-3-24/reference/modules/vectorstore.html)から利用できます。

FAISS ベクトルストアについてよりよく知るためには、こちらの[ドキュメント](https://arxiv.org/pdf/1702.08734.pdf)を参照してください。

In [ ]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.vectorstores import FAISS

s3_path = "s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/Amazon_SageMaker_FAQs.csv"
!aws s3 cp $s3_path ./rag_data/Amazon_SageMaker_FAQs.csv

loader = CSVLoader("./rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars, each row consists in a question column and an answer column
documents_aws = loader.load() #
print(f"Number of documents={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Number of documents after split and chunking={len(docs)}")
vectorstore_faiss_aws = None
try:
    
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = br_embeddings
    )

    print(f"vectorstore_faiss_aws: number of elements in the index={vectorstore_faiss_aws.index.ntotal}::")

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

#### セマンティック検索


LangChain が提供する Wrapper クラスを使用してベクトルデータベースストアをクエリし、関連するドキュメントを返すことができます。裏側では、これは RetrievalQA チェーンを実行するだけです。

In [ ]:
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print_ww(wrapper_store_faiss.query("R in SageMaker", llm=cl_llm))

セマンティック検索の仕組みを見てみましょう。

1. まず、クエリの埋め込みベクトルを計算します。
2. 次に、このベクトルを使用してストアの類似度検索を行います

In [ ]:
v = br_embeddings.embed_query("R in SageMaker")
print(v[0:10])
results = vectorstore_faiss_aws.similarity_search_by_vector(v, k=4)
for r in results:
    print_ww(r.page_content)
    print('----')

#### メモリ
どのチャットボットでも、ユースケースに応じてカスタマイズされたさまざまなオプションを備えた QA チェーンが必要になります。今回のチャットボットでは、モデルが回答を提供するために考慮する必要があるので、常に会話の履歴を保持しておく必要があります。この例では、LangChain の [ConversationalRetrievalChain](https://python.langchain.com/docs/modules/chains/popular/chat_vector_db) を ConversationBufferMemory と一緒に使用して会話の履歴を保存しています。

 `verbose` を `True` に設定すると、裏側で何が起こっているかをすべて確認することができます。

In [ ]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

print_ww(CONDENSE_QUESTION_PROMPT.template)

#### ConversationRetrievalChain に使用されるパラメータ

* **retriever**:  `VectorStore` をバックエンドとして使用した `VectorStoreRetriever` を使用しました。テキストを取得するには、`"similarity"` または `"mmr"` の 2 つの検索タイプを選択できます。`search_type="similarity"` の場合、Retriever オブジェクト内の類似検索を使用して、質問ベクトルに最も近いテキストチャンクベクトルを選択します。

* **memory**: 履歴を保存するためのメモリストアです。

* **condense_question_prompt**: ユーザーからの質問があった際、以前の会話とユーザーの質問を使用して、独立した質問を作成します。

* **chain_type**: チャット履歴が長く、コンテキストに合わない場合は、このパラメータを使用します。オプションは `stuff`, `refine`, `map_reduce`, `map-rerank` です。

質問が提供されたコンテキストの範囲外である場合、モデルはわからないと答えます
**注意**: チェーンがどのように機能するのかに興味がある場合は、`verbose=true` のコメントを外してください。

In [ ]:
# verbose を true に設定すると、ログとドキュメント全体が表示されます
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    memory=memory_chain,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    #verbose=True, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

それではチャットをしてみましょう！SageMakerについてのいくつかの質問をしてみます。
1. What is SageMaker?
2. What is canvas?

In [ ]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

状況によって異なるかもしれませんが、2、3回質問すると、奇妙な答えが出始めます。場合によっては、他の言語でさえ起こります。
この現象は、このノートブックの冒頭で説明したのと同じ理由で発生しています。デフォルトの LangChain プロンプトは Claude にとって最適ではないということです。 
次のセルに、2 つの新しいプロンプトを設定します。1 つは質問を書き換えるため、もう 1 つは言い換えた質問から回答を得るためのものです。

In [ ]:
# verbose を true に設定すると、ログとドキュメント全体が表示されます
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import BaseMessage


# また、履歴を Claude のチャットとして出力する別のチャット履歴取得機能も提供しています（例：\n\n を含む）
_ROLE_MAP = {"human": "\n\nHuman: ", "ai": "\n\nAssistant: "}
def _get_chat_history(chat_history):
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, BaseMessage):
            role_prefix = _ROLE_MAP.get(dialogue_turn.type, f"{dialogue_turn.type}: ")
            buffer += f"\n{role_prefix}{dialogue_turn.content}"
        elif isinstance(dialogue_turn, tuple):
            human = "\n\nHuman: " + dialogue_turn[0]
            ai = "\n\nAssistant: " + dialogue_turn[1]
            buffer += "\n" + "\n".join([human, ai])
        else:
            raise ValueError(
                f"Unsupported chat history format: {type(dialogue_turn)}."
                f" Full chat history: {chat_history} "
            )
    return buffer

# Claude のための圧縮プロンプト
condense_prompt_claude = PromptTemplate.from_template("""{chat_history}

Answer only with the new question.


Human: How would you ask the question considering the previous conversation: {question}


Assistant: Question:""")

# サンプリングするトークンを増やして Claude LLM を再作成します。これにより回答はより長くなりますが、ある程度の遅延が発生します
cl_llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={"max_tokens_to_sample": 500})
memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    #retriever=vectorstore_faiss_aws.as_retriever(search_type='similarity', search_kwargs={"k": 8}),
    memory=memory_chain,
    get_chat_history=_get_chat_history,
    #verbose=True,
    condense_question_prompt=condense_prompt_claude, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

# 回答取得のための LLMChain プロンプト。 ConversationalRetrievalChange は、コンストラクタでこのパラメータを公開しません。
qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
{context}

Human: Use at maximum 3 sentences to answer the question inside the <q></q> XML tags. 

<q>{question}</q>

Do not use any XML tags in the answer. If the answer is not in the context say "Sorry, I don't know as the answer was not found in the context"

Assistant:""")

それでは別のチャットを開始しましょう。このような質問をしてみましょう。

1. What is SageMaker?
2. what is canvas?

In [ ]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

#### プロンプトエンジニアリングを行う

プロンプトを「調整」して、多かれ少なかれ詳細な回答を得ることができます。たとえば、文章の数を変えたり、その命令をすべて削除したりしてみてください。完全な長さの答えを得るためには、`max_tokens_to_sample` の数 (1000 や 2000 など) を変更する必要があるかもしれません。

### このデモでは、Claude LLM を使用して次のパターンで会話型インターフェイスを作成しました。

1. チャットボット (基本 - コンテキストなし)

2. テンプレートを使用したチャットボット (Langchain)

3. ペルソナチャットボット

4. コンテキストを意識したチャットボット